# Delete a folder

In [ ]:
import shutil

F = 'output-v3' # Change this ONLY
folder_path = f'/content/{F}'

shutil.rmtree(folder_path)
print(f"Deleted folder: {folder_path}")

FileNotFoundError: [Errno 2] No such file or directory: '/content/output-v3'

# Stop warnings

In [ ]:
import warnings
warnings.filterwarnings("ignore")

# Start Profiling

In [ ]:
import cProfile
import pstats
import io

pr = cProfile.Profile()
pr.enable()

# Load YOLO12x

In [ ]:
!pip install ultralytics

In [ ]:
from ultralytics import YOLO

model = YOLO("yolo12x.pt")  # Use 'yolo12s.pt', 'n', 's', 'm', 'l', or 'x' for different sizes

# Imports

In [ ]:
!pip install deep_sort_realtime pillow numpy

In [ ]:
import os, cv2, numpy as np, argparse
from PIL import Image
from datetime import datetime
from deep_sort_realtime.deepsort_tracker import DeepSort
np.float = float  # compatibility fix for some old code

# Configurations (EDITABLE)

In [ ]:
V = "v2"
detection_interval = 1  # every frame is processed

# Video I/O Setup

In [ ]:
cap = cv2.VideoCapture(f"{V}.mp4")
os.makedirs(f"output-{V}", exist_ok=True)

timestamp = datetime.now().strftime("%Y%m%d-%H-%M-%S")
output_path = f"output-{V}/{V}-{timestamp}.mp4"

fourcc = cv2.VideoWriter_fourcc(*"mp4v")
fps = cap.get(cv2.CAP_PROP_FPS)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

# DeepSORT setup

In [ ]:
deepsort = DeepSort(max_age=30, n_init=3, nn_budget=100)

# Letterbox preprocessing function

In [ ]:
def letterbox_image(image, size=(640, 640)):
    ih, iw = image.shape[:2]
    h, w = size
    scale = min(w / iw, h / ih)
    nw, nh = int(iw * scale), int(ih * scale)

    # Resize
    image_resized = cv2.resize(image, (nw, nh))
    # Padding
    new_image = np.full((h, w, 3), 114, dtype=np.uint8)
    top = (h - nh) // 2
    left = (w - nw) // 2
    new_image[top:top+nh, left:left+nw] = image_resized

    return new_image, scale, left, top

# Main loop

In [ ]:
frame_count = 0
counted_ids = set()
track_lifetime = {}

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1
    print(f'{frame_count}/{total_frames}')

    detections = []

    if frame_count % detection_interval == 0:
        # Preprocess (letterbox + RGB)
        frame_resized, scale, pad_x, pad_y = letterbox_image(frame)
        rgb = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2RGB)

        # Predict
        results = model.predict(Image.fromarray(rgb))[0]

        print(f"Detected {len(results.boxes)} boxes")
        for box in results.boxes:
            cls = int(box.cls[0])
            conf = float(box.conf[0])
            if cls == 0 and conf > 0.25:  # person
                x1, y1, x2, y2 = box.xyxy[0]

                # Undo letterbox scaling
                x1 = (x1 - pad_x) / scale
                y1 = (y1 - pad_y) / scale
                x2 = (x2 - pad_x) / scale
                y2 = (y2 - pad_y) / scale

                x, y, w, h = x1, y1, x2 - x1, y2 - y1
                detections.append([(x, y, w, h), conf, 'person'])

    # === DEEP SORT UPDATE ===
    track_bbs_ids = deepsort.update_tracks(detections, frame=frame)
    print(f"DeepSORT got {len(track_bbs_ids)} tracks")

    # === DRAW DETECTIONS ===
    if len(detections) > 0:
        for det in detections:
            (x, y, w, h), conf, label = det
            x1, y1, x2, y2 = map(int, [x, y, x + w, y + h])
            shrink_factor = 0.9
            dw = int((x2 - x1) * (1 - shrink_factor) / 2)
            dh = int((y2 - y1) * (1 - shrink_factor) / 2)
            x1_s, y1_s, x2_s, y2_s = x1 + dw, y1 + dh, x2 - dw, y2 - dh

            overlay = frame.copy()
            cv2.rectangle(overlay, (x1_s, y1_s), (x2_s, y2_s), (255, 0, 0), -1)
            cv2.addWeighted(overlay, 0.4, frame, 0.6, 0, frame)

    # === DRAW TRACKS ===
    for track in track_bbs_ids:
        if not track.is_confirmed():
            continue
        track_id = track.track_id
        l, t, r, b = track.to_ltrb()
        x1, y1, x2, y2 = map(int, [l, t, r, b])

        track_lifetime[track_id] = track_lifetime.get(track_id, 0) + 1

        if track_lifetime[track_id] == 10 and track_id not in counted_ids:
            counted_ids.add(track_id)
            print(f"Track {track_id} confirmed. Total count: {len(counted_ids)}")

        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f'ID: {track_id}', (x1, y2 + 15),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    # === TOTAL COUNT ===
    total_text = f'Total: {len(counted_ids)}'
    (text_size, baseline) = cv2.getTextSize(total_text, cv2.FONT_HERSHEY_SIMPLEX, 0.8, 2)
    tw_total, th_total = text_size
    cv2.rectangle(frame, (10, 10), (10 + tw_total + 10, 10 + th_total + 10), (255, 250, 250), -1)
    cv2.putText(frame, total_text, (15, 10 + th_total + 5),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (125, 125, 125), 2)

    # === DETECTION COUNT ===
    detection_count_text = f'Detected: {len(detections)}'
    (text_size, baseline) = cv2.getTextSize(detection_count_text, cv2.FONT_HERSHEY_SIMPLEX, 0.8, 2)
    tw_det, th_det = text_size
    cv2.rectangle(frame, (width - tw_det - 20, 10), (width - 10, 10 + th_det + 10), (255, 250, 250), -1)
    cv2.putText(frame, detection_count_text, (width - tw_det - 15, 10 + th_det + 5),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (125, 125, 125), 2)

    out.write(frame)

1/289

0: 640x640 (no detections), 1679.8ms
Speed: 2.1ms preprocess, 1679.8ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
Detected 0 boxes
DeepSORT got 0 tracks
2/289

0: 640x640 (no detections), 1627.6ms
Speed: 2.0ms preprocess, 1627.6ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)
Detected 0 boxes
DeepSORT got 0 tracks
3/289

0: 640x640 (no detections), 1704.0ms
Speed: 2.0ms preprocess, 1704.0ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)
Detected 0 boxes
DeepSORT got 0 tracks
4/289

0: 640x640 (no detections), 1781.8ms
Speed: 1.7ms preprocess, 1781.8ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)
Detected 0 boxes
DeepSORT got 0 tracks
5/289

0: 640x640 1 person, 1498.7ms
Speed: 1.7ms preprocess, 1498.7ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 640)
Detected 1 boxes
DeepSORT got 1 tracks
6/289

0: 640x640 1 person, 1294.9ms
Speed: 1.9ms preprocess, 1294.9ms inference, 1.2ms postproce

# Output

In [ ]:
compatible_output_path = f"output-{V}/{V}-{timestamp}-whatsapp.mp4"
os.system(f'ffmpeg -y -i "{output_path}" -vcodec libx264 -profile:v baseline -level 3.0 -pix_fmt yuv420p -preset slow -crf 24 -movflags +faststart "{compatible_output_path}"')
print(f"WhatsApp-compatible video saved to: {compatible_output_path}")

WhatsApp-compatible video saved to: output-v2/v2-20250722-04-31-13-whatsapp.mp4


# Clean up

In [ ]:
cap.release()
out.release()
print(f"Saved output to: {output_path}")

Saved output to: output-v2/v2-20250722-04-31-13.mp4


# Finish Profiling

In [ ]:
pr.disable()
s = io.StringIO()
sortby = 'cumulative'  # or 'time'
ps = pstats.Stats(pr, stream=s).sort_stats(sortby)
ps.print_stats()
print(s.getvalue())

         7867606 function calls (7269755 primitive calls) in 433.591 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       18    0.001    0.000  433.860   24.103 /usr/lib/python3.11/asyncio/base_events.py:1859(_run_once)
      133    0.000    0.000  432.483    3.252 /usr/lib/python3.11/asyncio/events.py:82(_run)
      133    0.000    0.000  432.483    3.252 {method 'run' of '_contextvars.Context' objects}
        7    0.000    0.000  432.476   61.782 /usr/local/lib/python3.11/dist-packages/ipykernel/kernelbase.py:501(dispatch_queue)
       28    0.000    0.000  432.475   15.446 /usr/local/lib/python3.11/dist-packages/ipykernel/kernelbase.py:487(process_one)
       21    0.001    0.000  432.475   20.594 /usr/local/lib/python3.11/dist-packages/ipykernel/kernelbase.py:357(dispatch_shell)
       21    0.001    0.000  432.442   20.592 /usr/local/lib/python3.11/dist-packages/ipykernel/kernelbase.py:684(execute_request)
       